# COMP 6321 Machine Learning
## Assignment 1 Answers
### Name : [Parsa Kamalipour](https://benymaxparsa.github.io) , StudentID : 40310734

In [56]:
import os

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Exercise 2: Logistic Regression (9 pts)

In this exercise you will implement the logistic regression algorithm and evaluate it on the dataset provided
with this assignment. The training dataset is divided into five different csv files. You need to combine this
into a single training dataset. Do not use any machine learning library, but feel free to use libraries for linear
algebra and feel free to verify your results with existing machine learning libraries.

## Combine the Training Data:

In [57]:
directory = "logistic_regression-dataset"
test_x = None
test_y = None
data_x = None
data_y = None

for filename in os.listdir(directory):
    if filename.__eq__('testData.csv'):
        test_x = pd.read_csv(os.path.join(directory, filename), header=None)
    elif filename.__eq__('testLabels.csv'):
        test_y = pd.read_csv(os.path.join(directory, filename), header=None,)
    else:
        for i in range(1, 6):
             if filename.__eq__(f'trainData{i}.csv'):
                 if data_x is None:
                     data_x = pd.read_csv(os.path.join(directory, filename), header=None)
                 else:
                    data_x = pd.concat([data_x, pd.read_csv(os.path.join(directory, filename), header=None)], ignore_index=True)
        for i in range(1, 6):
            if filename.__eq__(f'trainLabels{i}.csv'):
                if data_y is None:
                    data_y = pd.read_csv(os.path.join(directory, filename), header=None,)
                else:
                    data_y = pd.concat([data_y, pd.read_csv(os.path.join(directory, filename), header=None)], ignore_index=True)


In [58]:
display(test_x, test_y)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,0,0,1,13,10,0,0,0,8,...,16,0,0,0,4,0,14,0,15,0
1,0,0,0,0,0,7,0,0,0,0,...,16,0,0,0,0,0,2,11,12,0
2,16,11,5,16,15,16,16,16,16,16,...,14,16,16,16,16,16,16,16,16,0
3,0,0,7,16,2,2,0,0,0,1,...,11,0,0,0,0,9,9,0,0,0
4,0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,13,14,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,1,6,0,0
106,0,0,0,8,2,12,0,0,7,0,...,6,0,0,0,5,10,6,5,0,0
107,16,14,16,16,16,16,7,16,0,16,...,6,10,16,9,16,16,16,10,9,16
108,0,0,0,0,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,0
0,6
1,6
2,5
3,6
4,6
...,...
105,6
106,5
107,5
108,6


In [59]:
display(data_x, data_y)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,0,11,0,16,0,0,0,4,0,...,0,0,0,0,0,0,3,5,0,0
1,0,0,9,16,16,16,0,0,0,0,...,5,0,3,0,15,13,10,3,0,16
2,9,16,15,10,16,16,0,0,16,13,...,16,16,0,16,16,7,16,16,16,3
3,0,0,0,6,0,1,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
4,0,5,0,14,16,0,0,0,0,0,...,15,0,0,3,0,9,0,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,0,0,16,16,9,15,0,0,4,7,...,0,7,0,2,14,9,13,0,0,1
497,0,0,0,6,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,1,4,10,16,13,15,12,1,7,12,...,5,2,8,0,0,5,11,0,0,0


,0
0,6
1,6
2,6
3,6
4,6
...,...
495,5
496,5
497,6
498,6


1) (2 pts) Let $\Pr(C_1|x) = \sigma({\bf{w}}^T{\bf{x}}+w_0)$ and $\Pr(C_2|{\bf{x}}) = 1 - \sigma({\bf{w}}^T{\bf{x}}+w_0)$. Learn the parameters ${\bf{w}}$ and $w_0$ using the gradient descent algorithm. Use the maximum number of epochs to be 100 for GD. You can also use any appropriate convergence criteria to stop the GD loop before 100 epochs. Use a step size of 0.1 (or 0.01 if it is converging poorly) for GD. 

## Initialize Parameters:

In [61]:
data_dimension = len(data_x.columns)
w_weight_vector = [0]*data_dimension
w0_bias = 0
learning_rate_large = 0.1
learning_rate_small = 0.01
max_epochs = 100

## Gradient Descent Algorithm: